In [1]:
import ndjson
import json
import random
import re
from functools import partial
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
lang = "julia"
with open(f"stack-code/{lang}/0000000.jsonl") as f: 
    ds = ndjson.load(f)

print("len: ", len(ds))

len:  100000


In [3]:
def print_ex(example): 
    print(example["max_stars_repo_name"])
    print(example["max_stars_repo_path"] + "\n" + "#"*40 + "\n")
    print(example["content"])

In [4]:
random.shuffle(ds)
for i in range(100):
    text = ds[i]
    print("STARTING EXAMPLE ", i)
    print_ex(text)
    print("\n" * 2)
    print("#" * 80)
    print("\n" * 2)

STARTING EXAMPLE  0
lucpaoli/Clapeyron.jl
src/Clapeyron.jl
########################################

module Clapeyron
using StaticArrays
using LinearAlgebra

#for the assoc solver
import PackedVectorsOfVectors
const PackedVofV = PackedVectorsOfVectors.PackedVectorOfVectors

#for non allocating vectors of zeros and ones
using FillArrays: FillArrays

using Roots: Roots
using NLSolvers
import Metaheuristics
using DiffResults, ForwardDiff
using Scratch 
include("solvers/Solvers.jl")
using .Solvers
using .Solvers: log, sqrt
using Unitful
import LogExpFunctions
include("constants.jl")
include("models/basetools.jl") #type hierarchy
include("database/ParamOptions.jl") 
include("utils/vectors.jl")
include("utils/ClapeyronParam.jl")


include("utils/macros.jl")
using CSV, Tables
include("database/database.jl")
include("utils/misc.jl")


include("models/combiningrules.jl")

include("models/eos.jl")
include("utils/visualisation.jl")
include("utils/split_model.jl")
include("database/UserReader.jl")

In [ ]:
def no_json_filter(example):
    return example["content"][0] != "{"

def filtered_out(filter):
    return [d for d in ds if not filter(d)]

filtered_out(no_json_filter)

In [ ]:
print(ds[0]["content"])

What we want for a Julia filter:
- Some files only include another file.